In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

In [3]:
# Load data
data = pd.read_csv('/content/drive/My Drive/M2 GL/PFE/Data/hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing_v2_2D.csv')

In [4]:
export_dir = '/content/drive/My Drive/M2 GL/PFE/new Tajweed rule model/AI models/v1'

In [5]:
abdul_basit = data[data['recitor_en'] == 'Abdul Basit']
yassin_aljazaery = data[data['recitor_en'] == 'Yassin Al Jazaery']
ibrahim_aldosary = data[data['recitor_en'] == 'Ibrahim_Aldosary']
al_husary = data[data['recitor_en'] == 'Al husary']

In [6]:
splitted_data_info_np = np.empty((0, 6))
models_information_np = np.empty((0, 5))

In [7]:
max_Y = {'madd_6_Lazim': 2, 'madd_246': 4, 'madd_6': 6, 'madd_2': 10, 'Ikhfaa': 8, 'Idgham': 18, 'tafkhim': 18, 'qalqala': 6, 'imala': 6}
max_X = 8000

In [8]:
def data_preparation(reciter_data, tajweed_rule):
  data_filtered = reciter_data[reciter_data[tajweed_rule].apply(lambda x: x != '[]')]

  # Extract 'mfcc' and tajweed_rule columns as lists of strings
  X_raw = data_filtered['mfcc'].astype(str).tolist()
  Y_raw = data_filtered[tajweed_rule].astype(str).tolist()

  # Preprocess the input data (X)
  X = [tf.constant(eval(x)) for x in X_raw]

  # Preprocess the target data (Y)
  Y = []
  for y_str in Y_raw:
      y_list = eval(y_str)
      # Flatten nested lists and convert to numpy array
      flat_list = [item for sublist in y_list for item in sublist]
      Y.append(flat_list)

  # Pad sequences in Y to ensure all have the same length
  Y_padded = tf.keras.preprocessing.sequence.pad_sequences(Y, maxlen=max_Y[tajweed_rule], padding='post', dtype='int32', value=-1)

  # Pad sequences in X to ensure all have the same length
  X_padded = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_X, padding='post', dtype='float32')

  # Split the data into training and testing sets
  X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y_padded, test_size=0.2, random_state=10)
  return X_train, X_test, Y_train, Y_test

In [9]:
def tajweed_rule_model(reciter1, reciter2, reciter3, reciter4, tajweed_rule):
  global splitted_data_info_np, models_information_np, data

  # data preparation
  reciter1_X_train, reciter1_X_test, reciter1_Y_train, reciter1_Y_test = data_preparation(reciter1, tajweed_rule)
  reciter2_X_train, reciter2_X_test, reciter2_Y_train, reciter2_Y_test = data_preparation(reciter2, tajweed_rule)
  reciter3_X_train, reciter3_X_test, reciter3_Y_train, reciter3_Y_test = data_preparation(reciter3, tajweed_rule)
  reciter4_X_train, reciter4_X_test, reciter4_Y_train, reciter4_Y_test = data_preparation(reciter4, tajweed_rule)

  # Update splitted_data_info with information about each reciter
  for reciter_X_train, reciter_X_test, reciter_Y_train, reciter_Y_test, reciter_data in [
      (reciter1_X_train, reciter1_X_test, reciter1_Y_train, reciter1_Y_test, reciter1),
      (reciter2_X_train, reciter2_X_test, reciter2_Y_train, reciter2_Y_test, reciter2),
      (reciter3_X_train, reciter3_X_test, reciter3_Y_train, reciter3_Y_test, reciter3),
      (reciter4_X_train, reciter4_X_test, reciter4_Y_train, reciter4_Y_test, reciter4)]:

      splitted_data_info_np = np.append(splitted_data_info_np, [[
              tajweed_rule,
              reciter_data.iloc[0]['recitor_en'],
              len(reciter_X_train),
              len(reciter_X_test),
              len(reciter_Y_train),
              len(reciter_Y_test)
              ]], axis=0)

  # concatenate data
  # training data
  X_train = np.concatenate([reciter1_X_train, reciter2_X_train, reciter3_X_train, reciter4_X_train], axis=0)
  Y_train = np.concatenate([reciter1_Y_train, reciter2_Y_train, reciter3_Y_train, reciter4_Y_train], axis=0)

  # testing data
  X_test = np.concatenate([reciter1_X_test, reciter2_X_test, reciter3_X_test, reciter4_X_test], axis=0)
  Y_test = np.concatenate([reciter1_Y_test, reciter2_Y_test, reciter3_Y_test, reciter4_Y_test], axis=0)

  splitted_data_info_np = np.append(splitted_data_info_np, [[
          tajweed_rule,
          'all reciters',
          len(X_train),
          len(X_test),
          len(Y_train),
          len(Y_test)
          ]], axis=0)

  # Normalize input data by scaling each sequence individually
  scaler = StandardScaler()
  X_train_scaled = np.array([scaler.fit_transform(seq) for seq in X_train])
  X_test_scaled = np.array([scaler.transform(seq) for seq in X_test])

  # Define a simple neural network model
  input_shape = X_train_scaled[0].shape  # Shape of each mfcc sequence
  output_shape = Y_train.shape[1]  # Dimension of output (number of units in output layer)

  input_layer = Input(shape=input_shape)
  flatten_layer = Flatten()(input_layer)  # Flatten the sequence to a 1D vector
  hidden_layer = Dense(64, activation='relu')(flatten_layer)
  output_layer = Dense(output_shape, activation='linear')(hidden_layer)

  model = Model(inputs=input_layer, outputs=output_layer)

  # Compile the model
  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, Y_train, epochs=50, batch_size=32, validation_split=0.1)

  #export the model
  model_filename = f'{tajweed_rule}_tajweed_rule_model'
  model_path = os.path.join(export_dir, model_filename)
  keras.models.save_model(model, model_path)

  # Make predictions on test data
  predictions = model.predict(X_test_scaled)

  # Evaluate the model with adjusted predictions
  predictions[predictions < 0] = -1
  predictions = np.round(predictions).astype('int32')
  loss, accuracy = model.evaluate(X_test_scaled, predictions)

  print(f"Test Loss: {loss:.4f}, Test accuracy : {accuracy:.4f}")
  models_information_np = np.append(models_information_np, [[
          model_filename,
          "{:.4f}".format(loss),
          "{:.4f}".format(accuracy),
          "{:.2f}".format(accuracy*100),
          model_path]], axis=0)

In [10]:
for rule in  max_Y.keys():
  tajweed_rule_model(abdul_basit, yassin_aljazaery, ibrahim_aldosary, al_husary, rule)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 217.8788 - accuracy: 0.3333 - val_loss: 35.4968 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 119ms/step - loss: 1620.6061 - accuracy: 0.3333 - val_loss: 44.1120 - val_accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 108ms/step - loss: 703.7656 - accuracy: 1.0000 - val_loss: 113.8120 - val_accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 200ms/step - loss: 53.9921 - accuracy: 0.6667 - val_loss: 156.1582 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 172ms/step - loss: 120.5444 - accuracy: 0.0000e+00 - val_loss: 177.4333 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 159ms/step - loss: 172.4748 - accuracy: 0.0000e+00 - val_loss: 180.7321 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 146ms/step - loss: 145.1547 - accuracy: 0.3333 - val_loss: 185.3

4/4 [==============================] - 0s 14ms/step - loss: 3.4517 - accuracy: 0.9333
Test Loss: 3.4517, Test accuracy : 0.9333
Epoch 1/50
13/13 [==============================] - 2s 105ms/step - loss: 255.7914 - accuracy: 0.2683 - val_loss: 289.6125 - val_accuracy: 0.3478
Epoch 2/50
13/13 [==============================] - 1s 91ms/step - loss: 173.5685 - accuracy: 0.3439 - val_loss: 250.2083 - val_accuracy: 0.3913
Epoch 3/50
13/13 [==============================] - 1s 92ms/step - loss: 137.8981 - accuracy: 0.2488 - val_loss: 275.9974 - val_accuracy: 0.1957
Epoch 4/50
13/13 [==============================] - 1s 92ms/step - loss: 118.4117 - accuracy: 0.2610 - val_loss: 255.5898 - val_accuracy: 0.2174
Epoch 5/50
13/13 [==============================] - 1s 90ms/step - loss: 104.6636 - accuracy: 0.2439 - val_loss: 256.7564 - val_accuracy: 0.3261
Epoch 6/50
13/13 [==============================] - 2s 121ms/step - loss: 97.4550 - accuracy: 0.3024 - val_loss: 247.2100 - val_accuracy: 0.2391
E

4/4 [==============================] - 0s 29ms/step - loss: 1.1440 - accuracy: 0.9483
Test Loss: 1.1440, Test accuracy : 0.9483
Epoch 1/50
21/21 [==============================] - 3s 100ms/step - loss: 196.4468 - accuracy: 0.2081 - val_loss: 233.7764 - val_accuracy: 0.2639
Epoch 2/50
21/21 [==============================] - 2s 87ms/step - loss: 148.8264 - accuracy: 0.2655 - val_loss: 233.7357 - val_accuracy: 0.1944
Epoch 3/50
21/21 [==============================] - 2s 104ms/step - loss: 114.4896 - accuracy: 0.2329 - val_loss: 214.8959 - val_accuracy: 0.2222
Epoch 4/50
21/21 [==============================] - 2s 113ms/step - loss: 103.1615 - accuracy: 0.3851 - val_loss: 215.1735 - val_accuracy: 0.2778
Epoch 5/50
21/21 [==============================] - 2s 104ms/step - loss: 91.0293 - accuracy: 0.3276 - val_loss: 189.5527 - val_accuracy: 0.2917
Epoch 6/50
21/21 [==============================] - 2s 82ms/step - loss: 88.8858 - accuracy: 0.3540 - val_loss: 208.0066 - val_accuracy: 0.2083


In [11]:
def print_model_summary(loaded_model, tajweed_rule):
  print(f'******* Tajweed rule {tajweed_rule} model *******')
  loaded_model.summary()
  print('\n')

In [12]:
for rule in max_Y.keys():
    model_filename = f'{rule}_tajweed_rule_model'
    model_path = os.path.join(export_dir, model_filename)

    # Load the saved model
    loaded_model = tf.keras.models.load_model(model_path)

    print_model_summary(loaded_model, rule)

******* Tajweed rule madd_6_Lazim model *******
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8000, 13)]        0         
                                                                 
 flatten (Flatten)           (None, 104000)            0         
                                                                 
 dense (Dense)               (None, 64)                6656064   
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 6656194 (25.39 MB)
Trainable params: 6656194 (25.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


******* Tajweed rule madd_246 model *******
Model: "model_1"
__________________________________________

In [13]:
# how data is splitted
columns1 = ['tajweed_rule', 'data_of', 'X_train_nb_samples', 'X_test_nb_samples', 'Y_train_nb_samples', 'X_test_nb_samples']
splitted_data_info = pd.DataFrame(data=splitted_data_info_np, columns=columns1)

# save models information
columns2 = ['Model', 'Loss', 'Accuracy', 'Accuracy %', 'Path_to_the_model']
models_information = pd.DataFrame(data=models_information_np, columns=columns2)

In [14]:
splitted_data_info

,tajweed_rule,data_of,X_train_nb_samples,X_test_nb_samples,Y_train_nb_samples,X_test_nb_samples
0,madd_6_Lazim,Abdul Basit,1,1,1,1
1,madd_6_Lazim,Yassin Al Jazaery,1,1,1,1
2,madd_6_Lazim,Ibrahim_Aldosary,1,1,1,1
3,madd_6_Lazim,Al husary,1,1,1,1
4,madd_6_Lazim,all reciters,4,4,4,4
5,madd_246,Abdul Basit,62,16,62,16
6,madd_246,Yassin Al Jazaery,62,16,62,16
7,madd_246,Ibrahim_Aldosary,62,16,62,16
8,madd_246,Al husary,62,16,62,16
9,madd_246,all reciters,248,64,248,64


In [15]:
models_information

,Model,Loss,Accuracy,Accuracy %,Path_to_the_model
0,madd_6_Lazim_tajweed_rule_model,225.4916,0.7500,75.00,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
1,madd_246_tajweed_rule_model,15.3457,1.0000,100.00,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
2,madd_6_tajweed_rule_model,115.5054,0.8750,87.50,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
3,madd_2_tajweed_rule_model,13.4253,0.9750,97.50,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
4,Ikhfaa_tajweed_rule_model,3.4517,0.9333,93.33,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
5,Idgham_tajweed_rule_model,1.1440,0.9483,94.83,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
6,tafkhim_tajweed_rule_model,50.6097,0.8778,87.78,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
7,qalqala_tajweed_rule_model,5739.4536,0.9875,98.75,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
8,imala_tajweed_rule_model,2.4608,0.9821,98.21,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
